# Assignment3.1 Use - Taxi Cancellations

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

import matplotlib.pylab as plt

from dmba import plotDecisionTree, gainsChart, liftChart
from dmba import classificationSummary, regressionSummary

%matplotlib inline

no display found. Using non-interactive Agg backend


In [2]:
# Load the data
df=pd.read_csv("Taxi-cancellation-case.csv")
df.head(10)

,row#,user_id,vehicle_model_id,package_id,travel_type_id,from_area_id,to_area_id,from_city_id,to_city_id,from_date,to_date,online_booking,mobile_site_booking,booking_created,from_lat,from_long,to_lat,to_long,Car_Cancellation
0,1,17712,12,NaN,2,1021.0,1323.0,NaN,NaN,1/1/13 22:33,NaN,0,0,1/1/13 8:01,13.028530,77.546250,12.869805,77.653211,0
1,2,17037,12,NaN,2,455.0,1330.0,NaN,NaN,1/1/13 12:43,NaN,0,0,1/1/13 9:59,12.999874,77.678120,12.953434,77.706510,0
2,3,761,12,NaN,2,814.0,393.0,NaN,NaN,1/2/13 0:28,1/3/13 0:00,1,0,1/1/13 12:14,12.908993,77.688900,13.199560,77.706880,0
3,4,868,12,NaN,2,297.0,212.0,NaN,NaN,1/1/13 13:12,NaN,0,0,1/1/13 12:42,12.997890,77.614880,12.994740,77.607970,0
4,5,21716,28,NaN,2,1237.0,330.0,NaN,NaN,1/1/13 16:33,NaN,0,0,1/1/13 15:07,12.926450,77.612060,12.858833,77.589127,0
5,6,38966,12,NaN,2,61.0,393.0,NaN,NaN,1/1/13 18:00,NaN,0,0,1/1/13 15:11,12.962980,77.712290,13.199560,77.706880,0
6,7,22196,12,NaN,2,409.0,1194.0,NaN,NaN,1/1/13 16:19,NaN,0,0,1/1/13 15:40,13.077460,77.606680,13.004460,77.569230,0
7,8,22200,12,NaN,2,1371.0,839.0,NaN,NaN,1/2/13 10:33,NaN,1,0,1/1/13 17:21,13.000418,77.674835,12.857730,77.786420,0
8,9,22201,12,1.0,3,1323.0,NaN,NaN,NaN,1/1/13 19:55,1/1/13 0:00,1,0,1/1/13 17:25,12.869805,77.653211,NaN,NaN,0
9,10,22202,12,NaN,2,1017.0,496.0,NaN,NaN,1/2/13 5:02,NaN,0,0,1/1/13 17:30,13.110840,77.600740,13.058450,77.640750,0


In [ ]:
# Basic information
print("shape of the dataset:", df.shape)
print("data types:", df.dtypes)

shape of the dataset: (10000, 19)
data types: row#                     int64
user_id                  int64
vehicle_model_id         int64
package_id             float64
travel_type_id           int64
from_area_id           float64
to_area_id             float64
from_city_id           float64
to_city_id             float64
from_date               object
to_date                 object
online_booking           int64
mobile_site_booking      int64
booking_created         object
from_lat               float64
from_long              float64
to_lat                 float64
to_long                float64
Car_Cancellation         int64
dtype: object


In [7]:
df.isnull().sum() #Check for missing values


row#                      0
user_id                   0
vehicle_model_id          0
package_id             8248
travel_type_id            0
from_area_id             15
to_area_id             2091
from_city_id           6294
to_city_id             9661
from_date                 0
to_date                4178
online_booking            0
mobile_site_booking       0
booking_created           0
from_lat                 15
from_long                15
to_lat                 2091
to_long                2091
Car_Cancellation          0
dtype: int64

In [6]:
df.describe()

,row#,user_id,vehicle_model_id,package_id,travel_type_id,from_area_id,to_area_id,from_city_id,to_city_id,online_booking,mobile_site_booking,from_lat,from_long,to_lat,to_long,Car_Cancellation
count,10000.00000,10000.000000,10000.000000,1752.000000,10000.000000,9985.000000,7909.000000,3706.000000,339.000000,10000.000000,10000.00000,9985.000000,9985.000000,7909.000000,7909.000000,10000.000000
mean,5000.50000,30663.966200,26.191700,1.988014,2.141300,709.767752,665.457833,14.898003,68.318584,0.353300,0.04240,12.983301,77.636423,13.026217,77.640752,0.074300
std,2886.89568,10988.599834,27.228053,1.401791,0.434917,418.968918,400.628562,1.190770,49.886427,0.478019,0.20151,0.085659,0.058667,0.113633,0.063522,0.262272
min,1.00000,16.000000,1.000000,1.000000,1.000000,2.000000,6.000000,1.000000,4.000000,0.000000,0.00000,12.776630,77.386930,12.776630,77.386930,0.000000
25%,2500.75000,24410.750000,12.000000,1.000000,2.000000,393.000000,393.000000,15.000000,32.000000,0.000000,0.00000,12.927320,77.594800,12.951850,77.587527,0.000000
50%,5000.50000,31510.000000,12.000000,2.000000,2.000000,590.000000,516.000000,15.000000,49.000000,0.000000,0.00000,12.969368,77.635750,12.982750,77.645030,0.000000
75%,7500.25000,39094.500000,24.000000,2.000000,2.000000,1086.000000,1052.000000,15.000000,108.000000,1.000000,0.00000,13.006450,77.687790,13.199560,77.706880,0.000000
max,10000.00000,48729.000000,91.000000,7.000000,3.000000,1401.000000,1403.000000,15.000000,203.000000,1.000000,1.00000,13.366072,77.786420,13.366072,77.786420,1.000000


## 1. 

## 2. 

## 3. 

### 3.1

### 3.2

### 3.3

### 3.4

## 4. 

## 5.

## 6.

## 7.